In [1]:
import sys, math, urllib.request, io, operator
from tile_system import *
from PIL import Image


In [2]:
def getURL(quadkey):
        license_key = "Aq3bmbTehDPHWHjRsSsw1oKi3z5bYUewA4t0LQp09ZU8HbVvXbW2HXRiFwshMf8h"
        return "http://h0.ortho.tiles.virtualearth.net/tiles/h%s.jpeg?g=131&key=%s" % (quadkey, license_key)

In [3]:
def getImageFromQuadkey(quadkey):
        socket = urllib.request.urlopen(getURL(quadkey))
        print('d')
        tinker = socket.read()
        img = Image.open(io.BytesIO(tinker))
        print('sucessiful')
        return img

def getLowestQuality(lat1, lon1, lat2, lon2):
        '''
        Here we search for the lowest Level of Detail that contains minimum number of tiles.
        '''
        i = 1
        for i in range(23, 0, -1):
                tx1, ty1 = LatLongToTileXY(lat1, lon1, i)
                tx2, ty2 = LatLongToTileXY(lat2, lon2, i)
                if tx1 > tx2:
                        tx1, tx2 = tx2, tx1
                if ty1 > ty2:
                        ty1, ty2 = ty2, ty1
                if (tx2 - tx1 <= 1) and (ty2 - ty1 <= 1):
                        print ("Lowest Quality found at "),
                        print (i)
                        return i, tx1, ty1

        
def nullResult(img):
        '''Compare the query result with the null image. If they match, then Bing server does not have data for that location'''
        result = (img == Image.open('null.jpeg'))
        print ('null')
        return result

In [5]:
def main():
        lat1 = float(sys.argv[1])
        lon1 = float(sys.argv[2])
        lat2 = float(sys.argv[3])
        lon2 = float(sys.argv[4])

        framework = 4096
        ministemp = 128
        base = Image.new('RGB', (framework, framework))

        quality, tx0, ty0 = getLowestQuality(lat1, lon1, lat2, lon2)
        curr_tile_size = int(framework/2)
        flag = True
        q=quality
        # we use a while loop and refresh the whole image for lowest quality to the best we can get. and pass the recent base quality image to best
        best_quality = base 
        while q <= 23 and curr_tile_size >= ministemp and flag:

                print ("Current level of detials ",)
                print (q)
                print ("curr_tile_size is",)
                print (curr_tile_size)
                tx1, ty1 = LatLongToTileXY(lat1, lon1, q)
                tx2, ty2 = LatLongToTileXY(lat2, lon2, q)
                if tx1 > tx2:
                        tx1, tx2 = tx2, tx1
                if ty1 > ty2:
                        ty1, ty2 = ty2, ty1
                for x in range(tx1, tx2+1):
                        if not flag:
                                break
                        for y in range(ty1, ty2+1):
                                if not flag:
                                        break
                                curr_quadKey = TileXYToQuadKey(x, y, q)
                                print ("curr_quadKey is",)
                                print (curr_quadKey)

                                try:
                                        curr_image = getImageFromQuadkey(curr_quadKey)

                                        if not nullResult(curr_image):
                                                curr_image = curr_image.resize((curr_tile_size, curr_tile_size))
                                                start_x = (x - tx0) * curr_tile_size 
                                                start_y = (y - ty0) * curr_tile_size
                                                end_x = start_x + curr_tile_size
                                                end_y = start_y + curr_tile_size
                                                base.paste(curr_image, (start_x, start_y, end_x, end_y))
                                        else:
                                                flag = False
                                                print("can't find tile:",)

                                                break
                                except:
                                        print('time out in connecting to server or cannot find more detailed image')
                                        flag =False
                                        break


                tx0 = tx0 * 2
                ty0 = ty0 * 2
                q += 1
                curr_tile_size= int(curr_tile_size/2)
                if flag:
                        best_quality = base
        print ("Refinary is finished")
        px1,py1=LatLongToPixelXY(lat1, lon1, quality)
        px2,py2=LatLongToPixelXY(lat2, lon2, quality)
        print(px1,py1)
        print(px2,py2)
        px1d=8*(px1 % 256)
        py1d=8*(py1 % 256)
        px2d=8*(px2 % 256)
        py2d=8*(py2 % 256)
        print(px1d,py1d)
        print(px2d,py2d)
        raw = best_quality.load()
        for aa in range(4096-1):
                for bb in range(4096-1):
                        if aa<px1d or aa>px2d or bb<py1d or bb>py2d:
                                raw[aa,bb]=(250,250,250)
                        
        print ("windowing finished")
        best_quality.save('complete.jpeg')


        

if __name__ == '__main__':
        main()


SyntaxError: invalid syntax (<ipython-input-5-6943165926bb>, line 90)